In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
%run
"./include/path_folder"

In [0]:
yt_schema = StructType(fields = [StructField("vdo_id", StringType(), False),
                                    StructField('vdo_uploader', StringType(),True),
                                    StructField("vdo_interval", DoubleType(), True),
                                    StructField("vdo_category", StringType(), True),
                                    StructField("vdo_length", DoubleType(), True),
                                    StructField("vdo_views", DoubleType(), True),
                                    StructField("vdo_rating", DoubleType(), True),
                                    StructField("vdo_num_rating", DoubleType(), True),
                                    StructField("vdo_comm", DoubleType(), True),
                                    StructField("vdo_rel_id", StringType(), True)])

In [0]:
yt_df = spark.read.option('Header', True).schema(yt_schema).csv(f'{raw_path_folder}/YouTube')

In [0]:
yt_df.show()

+-----------+--------------------+------------+----------------+----------+---------+----------+--------------+--------+-----------+
 vdo_id| vdo_uploader|vdo_interval| vdo_category|vdo_length|vdo_views|vdo_rating|vdo_num_rating|vdo_comm| vdo_rel_id|
+-----------+--------------------+------------+----------------+----------+---------+----------+--------------+--------+-----------+
QuRYeRnAuXM|EvilSquirrelPictures| 1135.0| Pets & Animals| 252.0| 1075.0| 4.96| 46.0| 86.0|gFa1YMEJFag|
3TYqkBJ9YRk| hggh22| 1135.0| Comedy| 169.0| 228.0| 5.0| 5.0| 3.0|QuRYeRnAuXM|
rSJ8QZWBegU| TimeGem| 1135.0| Entertainment| 95.0| 356.0| 4.31| 13.0| 1.0|QuRYeRnAuXM|
nRcovJn9xHg| wooochacha| 1135.0| Entertainment| 118.0| 1115.0| 2.23| 57.0| 73.0|QuRYeRnAuXM|
UEvVksP91kg| johnx113| 1135.0| Entertainment| 83.0| 281.0| 2.67| 9.0| 16.0|gFa1YMEJFag|
sVkuOk4jmCo| marcosoycadiz| 1135.0| People & Blogs| 49.0| 1196.0| 3.67| 6.0| 5.0|yjt5v88-6Aw|
ZTopArY7Nbg| TroyChannel| 1136.0| Entertainment| 145.0| 11.0| 0.0| 0.0| 0.0|THM3FYUNyr8|
0RViGi2Rne8| OmnipotentSI| 1135.0| Entertainment| 9.0| 490.0| 2.31| 26.0| 15.0|rSJ8QZWBegU|
HT_QlOJbDpg| dumbasstheredneck| 1095.0| Comedy| 14.0| 1349.0| 2.09| 22.0| 13.0|cdbp6rCVzRs|
ShhClb6J-NA| thecrashguy| 1136.0| Entertainment| 61.0| 29.0| 0.0| 0.0| 0.0|8dEezTw8J_o|
g9e1alirMhc| iamdanielisaac| 541.0| Comedy| 152.0| 483053.0| 4.65| 2689.0| 2932.0|_i6OD_y_-AI|
YZev1imoxX8| threefirstnames| 1135.0| Entertainment| 253.0| 63.0| 5.0| 3.0| 2.0|ldZPDf2cwqI|
I4yKEK9o8gA| djlogitek| 964.0| News & Politics| 173.0| 106929.0| 4.39| 1010.0| 1484.0|g9e1alirMhc|
1GKaVzNDbuI| fatvids| 690.0| Comedy| 176.0|2464486.0| 4.5| 13342.0| 11997.0|Bd61LnxqSng|
yuZhwV24PmM| weneedhelp| 1082.0| News & Politics| 213.0| 46183.0| 4.71| 280.0| 247.0|hU9iCANi02o|
zQ83d_D2MGs| dumbasstheredneck| 1095.0| Comedy| 14.0| 1550.0| 1.57| 14.0| 4.0|HT_QlOJbDpg|
DomumdGQSg8| imgrown18| 910.0| Comedy| 224.0| 392908.0| 4.8| 1261.0| 829.0|lVTS0iuo1EY|
pFUYi7dp1WU| StanleyJenkins| 1106.0| Comedy| 200.0| 110272.0| 4.07| 1011.0| 482.0|_1haYY5pV18|
2l6vwAIAqNU| makemagazine| 460.0|Film & Animation| 294.0| 71966.0| 4.07| 260.0| 229.0|g9e1alirMhc|
hiSmlmXp-aU| bravenewfilms| 440.0| Entertainment| 168.0| 653840.0| 4.43| 198.0| 377.0|ED6YaRl93CE|
+-----------+--------------------+------------+----------------+----------+---------+----------+--------------+--------+-----------+
only showing top 20 rows

In [0]:
rank_spec = yt_df.groupBy('vdo_category').agg(count('vdo_id')).orderBy(desc('count(vdo_id)')).withColumnRenamed('count(vdo_id)', 'Number_of_videos')

In [0]:
rank_prep = Window.orderBy(desc("Number_of_videos"))
rank_df = rank_spec.withColumn("Rank",rank().over(rank_prep))

In [0]:
display(rank_df)

vdo_category,Number_of_videos,Rank
Entertainment,908,1
Music,862,2
Comedy,414,3
People & Blogs,398,4
News & Politics,333,5
Film & Animation,260,6
Sports,251,7
Howto & Style,137,8
Travel & Events,112,9
Pets & Animals,95,10


In [0]:
vdo_ratings_df = yt_df.orderBy(desc('vdo_rating'),desc('vdo_views'),desc('vdo_num_rating'))
display(vdo_ratings_df)

vdo_id,vdo_uploader,vdo_interval,vdo_category,vdo_length,vdo_views,vdo_rating,vdo_num_rating,vdo_comm,vdo_rel_id
nUDksaQp3IA,kinxero,952.0,Entertainment,559.0,101016.0,5.0,4.0,81.0,8yMs-nvci18
K4i7PEClSF8,angelofevil69,691.0,Entertainment,238.0,75573.0,5.0,4.0,588.0,dLxWIlxDiRY
nwqXVAYsYW4,employeofdayear,1122.0,Music,291.0,28966.0,5.0,19.0,16.0,krqCi_y3z1g
hnA0VyR3HI4,u3astside,1107.0,Entertainment,373.0,22921.0,5.0,37.0,62.0,#NAME?
od1PLcqUhfs,badboyforeverdotcom,1040.0,Music,312.0,22118.0,5.0,38.0,19.0,KN_ACKracDU
Yx2iza6EfZo,taperted5150,781.0,Music,641.0,21049.0,5.0,8.0,1.0,null
8o72MzrgFiU,ReelNASA,1087.0,Autos & Vehicles,599.0,17824.0,5.0,40.0,41.0,78ChatsRe6Y
gjbG5N4EfDc,ruitico34,838.0,Music,261.0,14752.0,5.0,43.0,23.0,BuNLvd77hBc
kXeamc7e31c,kipkay,1095.0,Howto & Style,72.0,14413.0,5.0,26.0,21.0,kB_gwYz5Mqs
MIQFTBsGccA,HortonGB,1116.0,Film & Animation,174.0,13103.0,5.0,42.0,45.0,TEJenuU1QKE


In [0]:
rank_df.write.mode("overwrite").partitionBy("vdo_category").parquet (f"{processed_path_folder}/YouTube/Rank")

In [0]:
vdo_ratings_df.write.mode("overwrite").partitionBy("vdo_category").parquet (f"{processed_path_folder}/YouTube/Ratings")

In [0]:
yt = spark.read.parquet(f"{processed_path_folder}/YouTube/Rank").orderBy(asc('Rank'))
display(yt)

Number_of_videos,Rank,vdo_category
908,1,Entertainment
862,2,Music
414,3,Comedy
398,4,People & Blogs
333,5,News & Politics
260,6,Film & Animation
251,7,Sports
137,8,Howto & Style
112,9,Travel & Events
95,10,Pets & Animals


In [0]:
yt_ratings = spark.read.parquet(f"{processed_path_folder}/YouTube/Ratings")
display(yt_ratings)

vdo_id,vdo_uploader,vdo_interval,vdo_length,vdo_views,vdo_rating,vdo_num_rating,vdo_comm,vdo_rel_id,vdo_category
nUDksaQp3IA,kinxero,952.0,559.0,101016.0,5.0,4.0,81.0,8yMs-nvci18,Entertainment
K4i7PEClSF8,angelofevil69,691.0,238.0,75573.0,5.0,4.0,588.0,dLxWIlxDiRY,Entertainment
hnA0VyR3HI4,u3astside,1107.0,373.0,22921.0,5.0,37.0,62.0,#NAME?,Entertainment
acG4teg-uX4,AsikTV,957.0,184.0,12513.0,5.0,16.0,6.0,8-QZSkoYxaw,Entertainment
XSAzJP8H30o,blindi1,947.0,161.0,9950.0,5.0,9.0,7.0,WdtIUTIU--w,Entertainment
OSJjtP_p-Hs,SamTubeCOM,1127.0,306.0,9433.0,5.0,1.0,31.0,zbr1EiIC2Ro,Entertainment
q-JndN8vrrE,badboy4lyfe2,968.0,366.0,9357.0,5.0,32.0,28.0,wDKaX0xkUmE,Entertainment
Hg47-CwiP-I,AsikTV,973.0,566.0,8690.0,5.0,25.0,3.0,mGE6diUx2IE,Entertainment
EohHqWBlx9Y,jani331691,794.0,469.0,7907.0,5.0,15.0,21.0,MxnyorU_Fr8,Entertainment
dISPc5wvX64,MadeinchinaReview,1111.0,225.0,7846.0,5.0,1.0,17.0,L30ePvCD8Ls,Entertainment


In [0]:
%sql
create database youtube;

In [0]:
youtube = spark.read.parquet(f"{processed_path_folder}/YouTube/Ratings")

In [0]:
display(youtube)

vdo_id,vdo_uploader,vdo_interval,vdo_length,vdo_views,vdo_rating,vdo_num_rating,vdo_comm,vdo_rel_id,vdo_category
nUDksaQp3IA,kinxero,952.0,559.0,101016.0,5.0,4.0,81.0,8yMs-nvci18,Entertainment
K4i7PEClSF8,angelofevil69,691.0,238.0,75573.0,5.0,4.0,588.0,dLxWIlxDiRY,Entertainment
hnA0VyR3HI4,u3astside,1107.0,373.0,22921.0,5.0,37.0,62.0,#NAME?,Entertainment
acG4teg-uX4,AsikTV,957.0,184.0,12513.0,5.0,16.0,6.0,8-QZSkoYxaw,Entertainment
XSAzJP8H30o,blindi1,947.0,161.0,9950.0,5.0,9.0,7.0,WdtIUTIU--w,Entertainment
OSJjtP_p-Hs,SamTubeCOM,1127.0,306.0,9433.0,5.0,1.0,31.0,zbr1EiIC2Ro,Entertainment
q-JndN8vrrE,badboy4lyfe2,968.0,366.0,9357.0,5.0,32.0,28.0,wDKaX0xkUmE,Entertainment
Hg47-CwiP-I,AsikTV,973.0,566.0,8690.0,5.0,25.0,3.0,mGE6diUx2IE,Entertainment
EohHqWBlx9Y,jani331691,794.0,469.0,7907.0,5.0,15.0,21.0,MxnyorU_Fr8,Entertainment
dISPc5wvX64,MadeinchinaReview,1111.0,225.0,7846.0,5.0,1.0,17.0,L30ePvCD8Ls,Entertainment


In [0]:
youtube.write.format("parquet").saveAsTable("youtube.youtubedata")